# Library

In [ ]:
# !pip install arch
# !pip install tqdm

In [1]:
import os
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
import itertools
from tqdm import tqdm

import statsmodels.api as sm
import arch

# Logic Generator for Extracting of Data
**[스키마: 구단위 레벨로 지수 존재]**  
전국 수도권(서울/경기/인천) 6대광역시(부산/대구/광주/대전/울산/인천) 5대광역시(부산/대구/광주/대전/울산) 기타지방(나머지)  
서울 강남 강북 경기 인천 부산 대구 광주 대전 울산  
강원 충북 충남 전북 전남 경북 경남 제주  

**[선별: 구단위 제외한 지수 중 전국지역]**  
서울 강북 강남 6대광역시 경기 강원 충북 충남 전북 전남 경북 경남 (제주) 기타지방(수도권광역시제외지역)  

**[선별: 구단위 제외한 값으로 아파트/단독/연립 모두 존재 지역]**  
서울 강북 강남 6개광역시 경기 기타지방(수도권광역시제외지역)  
(강원 충북 충남 전북 전남 경북 경남은 아파트만 값 존재)  

In [2]:
# Calculate Logic of Data Extraction from KB Housing File
def Extracting_IndexValue(type_names, region_names, sheet_names, selected_names, sheet_val, col_val):
    col_names = ['Time']
    for combination in itertools.product(region_names, type_names):
        col_names.append(combination[0]+':'+combination[1])

    col_logic = ['='+sheet_names[0]+'!A5']
    for combination in itertools.product(selected_names, sheet_names):
        col_logic.append('='+combination[1]+'!'+combination[0])
    index_val = pd.DataFrame(col_logic, index=col_names).T

    val_index = []
    for combination in itertools.product(sheet_val, col_val):
        val_index.append('='+combination[0]+'!'+combination[1])
    value = pd.DataFrame(val_index, index=col_names[1:]).T
    return index_val, value

type_names = ['Apartments(HPI)', 'Detached_Houses(HPI)', 'Row_Houses(HPI)']
region_names = ['Seoul', 'Northern Seoul', 'Southern Seoul', '6 Large Cities', 'Gyeonggi-do', 'Gangwon-do', 'Chungcheongbuk-do', 'Chungcheongnam-do', 
                'Jeollabuk-do', 'Jeollanam-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Non-Metropolitan Area']
sheet_names = ['매매APT', '매매단독', '매매연립']
selected_names = ['C5', 'D5', 'S5', 'AE5', 'CJ5', 'EE5', 'EI5', 'EQ5', 'FA5', 'FG5', 'FL5', 'FS5', 'GG5']
sheet_val = ['평균매매']
col_val = ['I125', 'J125', 'K125', 'M125', 'N125', 'O125', 'Q125', 'R125', 'S125', 'U125', 'V125', 'W125', 'AU125', 'AV125', 'AW125', 
               'AY125', '_125', '_125', 'BA125', '_125', '_125', 'BC125', '_125', '_125', 'BE125', '_125', '_125', 'BG125', '_125', '_125', 
               'BI125', '_125', '_125', 'BK125', '_125', '_125', 'BM125', 'BN125', 'BO125']
HPI_index, HPI_value = Extracting_IndexValue(type_names, region_names, sheet_names, selected_names, sheet_val, col_val)

type_names = ['Apartments(JDI)', 'Detached_Houses(JDI)', 'Row_Houses(JDI)']
region_names = ['Seoul', 'Northern Seoul', 'Southern Seoul', '6 Large Cities', 'Gyeonggi-do', 'Gangwon-do', 'Chungcheongbuk-do', 'Chungcheongnam-do', 
                'Jeollabuk-do', 'Jeollanam-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Non-Metropolitan Area']
sheet_names = ['전세APT', '전세단독', '전세연립']
selected_names = ['C5', 'D5', 'S5', 'AE5', 'CJ5', 'EE5', 'EI5', 'EQ5', 'FA5', 'FG5', 'FL5', 'FS5', 'GG5']
sheet_val = ['평균전세']
col_val = ['I95', 'J95', 'K95', 'M95', 'N95', 'O95', 'Q95', 'R95', 'S95', 'U95', 'V95', 'W95', 'AU95', 'AV95', 'AW95', 
               'AY95', '_95', '_95', 'BA95', '_95', '_95', 'BC95', '_95', '_95', 'BE95', '_95', '_95', 'BG95', '_95', '_95', 
               'BI95', '_95', '_95', 'BK95', '_95', '_95', 'BM95', 'BN95', 'BO95']
JDI_index, JDI_value = Extracting_IndexValue(type_names, region_names, sheet_names, selected_names, sheet_val, col_val)
display(HPI_index)
display(HPI_value)
display(JDI_index)
display(JDI_value)

,Time,Seoul:Apartments(HPI),Seoul:Detached_Houses(HPI),Seoul:Row_Houses(HPI),Northern Seoul:Apartments(HPI),Northern Seoul:Detached_Houses(HPI),Northern Seoul:Row_Houses(HPI),Southern Seoul:Apartments(HPI),Southern Seoul:Detached_Houses(HPI),Southern Seoul:Row_Houses(HPI),6 Large Cities:Apartments(HPI),6 Large Cities:Detached_Houses(HPI),6 Large Cities:Row_Houses(HPI),Gyeonggi-do:Apartments(HPI),Gyeonggi-do:Detached_Houses(HPI),Gyeonggi-do:Row_Houses(HPI),Gangwon-do:Apartments(HPI),Gangwon-do:Detached_Houses(HPI),Gangwon-do:Row_Houses(HPI),Chungcheongbuk-do:Apartments(HPI),Chungcheongbuk-do:Detached_Houses(HPI),Chungcheongbuk-do:Row_Houses(HPI),Chungcheongnam-do:Apartments(HPI),Chungcheongnam-do:Detached_Houses(HPI),Chungcheongnam-do:Row_Houses(HPI),Jeollabuk-do:Apartments(HPI),Jeollabuk-do:Detached_Houses(HPI),Jeollabuk-do:Row_Houses(HPI),Jeollanam-do:Apartments(HPI),Jeollanam-do:Detached_Houses(HPI),Jeollanam-do:Row_Houses(HPI),Gyeongsangbuk-do:Apartments(HPI),Gyeongsangbuk-do:Detached_Houses(HPI),Gyeongsangbuk-do:Row_Houses(HPI),Gyeongsangnam-do:Apartments(HPI),Gyeongsangnam-do:Detached_Houses(HPI),Gyeongsangnam-do:Row_Houses(HPI),Non-Metropolitan Area:Apartments(HPI),Non-Metropolitan Area:Detached_Houses(HPI),Non-Metropolitan Area:Row_Houses(HPI)
0,=매매APT!A5,=매매APT!C5,=매매단독!C5,=매매연립!C5,=매매APT!D5,=매매단독!D5,=매매연립!D5,=매매APT!S5,=매매단독!S5,=매매연립!S5,=매매APT!AE5,=매매단독!AE5,=매매연립!AE5,=매매APT!CJ5,=매매단독!CJ5,=매매연립!CJ5,=매매APT!EE5,=매매단독!EE5,=매매연립!EE5,=매매APT!EI5,=매매단독!EI5,=매매연립!EI5,=매매APT!EQ5,=매매단독!EQ5,=매매연립!EQ5,=매매APT!FA5,=매매단독!FA5,=매매연립!FA5,=매매APT!FG5,=매매단독!FG5,=매매연립!FG5,=매매APT!FL5,=매매단독!FL5,=매매연립!FL5,=매매APT!FS5,=매매단독!FS5,=매매연립!FS5,=매매APT!GG5,=매매단독!GG5,=매매연립!GG5


,Seoul:Apartments(HPI),Seoul:Detached_Houses(HPI),Seoul:Row_Houses(HPI),Northern Seoul:Apartments(HPI),Northern Seoul:Detached_Houses(HPI),Northern Seoul:Row_Houses(HPI),Southern Seoul:Apartments(HPI),Southern Seoul:Detached_Houses(HPI),Southern Seoul:Row_Houses(HPI),6 Large Cities:Apartments(HPI),6 Large Cities:Detached_Houses(HPI),6 Large Cities:Row_Houses(HPI),Gyeonggi-do:Apartments(HPI),Gyeonggi-do:Detached_Houses(HPI),Gyeonggi-do:Row_Houses(HPI),Gangwon-do:Apartments(HPI),Gangwon-do:Detached_Houses(HPI),Gangwon-do:Row_Houses(HPI),Chungcheongbuk-do:Apartments(HPI),Chungcheongbuk-do:Detached_Houses(HPI),Chungcheongbuk-do:Row_Houses(HPI),Chungcheongnam-do:Apartments(HPI),Chungcheongnam-do:Detached_Houses(HPI),Chungcheongnam-do:Row_Houses(HPI),Jeollabuk-do:Apartments(HPI),Jeollabuk-do:Detached_Houses(HPI),Jeollabuk-do:Row_Houses(HPI),Jeollanam-do:Apartments(HPI),Jeollanam-do:Detached_Houses(HPI),Jeollanam-do:Row_Houses(HPI),Gyeongsangbuk-do:Apartments(HPI),Gyeongsangbuk-do:Detached_Houses(HPI),Gyeongsangbuk-do:Row_Houses(HPI),Gyeongsangnam-do:Apartments(HPI),Gyeongsangnam-do:Detached_Houses(HPI),Gyeongsangnam-do:Row_Houses(HPI),Non-Metropolitan Area:Apartments(HPI),Non-Metropolitan Area:Detached_Houses(HPI),Non-Metropolitan Area:Row_Houses(HPI)
0,=평균매매!I125,=평균매매!J125,=평균매매!K125,=평균매매!M125,=평균매매!N125,=평균매매!O125,=평균매매!Q125,=평균매매!R125,=평균매매!S125,=평균매매!U125,=평균매매!V125,=평균매매!W125,=평균매매!AU125,=평균매매!AV125,=평균매매!AW125,=평균매매!AY125,=평균매매!_125,=평균매매!_125,=평균매매!BA125,=평균매매!_125,=평균매매!_125,=평균매매!BC125,=평균매매!_125,=평균매매!_125,=평균매매!BE125,=평균매매!_125,=평균매매!_125,=평균매매!BG125,=평균매매!_125,=평균매매!_125,=평균매매!BI125,=평균매매!_125,=평균매매!_125,=평균매매!BK125,=평균매매!_125,=평균매매!_125,=평균매매!BM125,=평균매매!BN125,=평균매매!BO125


,Time,Seoul:Apartments(JDI),Seoul:Detached_Houses(JDI),Seoul:Row_Houses(JDI),Northern Seoul:Apartments(JDI),Northern Seoul:Detached_Houses(JDI),Northern Seoul:Row_Houses(JDI),Southern Seoul:Apartments(JDI),Southern Seoul:Detached_Houses(JDI),Southern Seoul:Row_Houses(JDI),6 Large Cities:Apartments(JDI),6 Large Cities:Detached_Houses(JDI),6 Large Cities:Row_Houses(JDI),Gyeonggi-do:Apartments(JDI),Gyeonggi-do:Detached_Houses(JDI),Gyeonggi-do:Row_Houses(JDI),Gangwon-do:Apartments(JDI),Gangwon-do:Detached_Houses(JDI),Gangwon-do:Row_Houses(JDI),Chungcheongbuk-do:Apartments(JDI),Chungcheongbuk-do:Detached_Houses(JDI),Chungcheongbuk-do:Row_Houses(JDI),Chungcheongnam-do:Apartments(JDI),Chungcheongnam-do:Detached_Houses(JDI),Chungcheongnam-do:Row_Houses(JDI),Jeollabuk-do:Apartments(JDI),Jeollabuk-do:Detached_Houses(JDI),Jeollabuk-do:Row_Houses(JDI),Jeollanam-do:Apartments(JDI),Jeollanam-do:Detached_Houses(JDI),Jeollanam-do:Row_Houses(JDI),Gyeongsangbuk-do:Apartments(JDI),Gyeongsangbuk-do:Detached_Houses(JDI),Gyeongsangbuk-do:Row_Houses(JDI),Gyeongsangnam-do:Apartments(JDI),Gyeongsangnam-do:Detached_Houses(JDI),Gyeongsangnam-do:Row_Houses(JDI),Non-Metropolitan Area:Apartments(JDI),Non-Metropolitan Area:Detached_Houses(JDI),Non-Metropolitan Area:Row_Houses(JDI)
0,=전세APT!A5,=전세APT!C5,=전세단독!C5,=전세연립!C5,=전세APT!D5,=전세단독!D5,=전세연립!D5,=전세APT!S5,=전세단독!S5,=전세연립!S5,=전세APT!AE5,=전세단독!AE5,=전세연립!AE5,=전세APT!CJ5,=전세단독!CJ5,=전세연립!CJ5,=전세APT!EE5,=전세단독!EE5,=전세연립!EE5,=전세APT!EI5,=전세단독!EI5,=전세연립!EI5,=전세APT!EQ5,=전세단독!EQ5,=전세연립!EQ5,=전세APT!FA5,=전세단독!FA5,=전세연립!FA5,=전세APT!FG5,=전세단독!FG5,=전세연립!FG5,=전세APT!FL5,=전세단독!FL5,=전세연립!FL5,=전세APT!FS5,=전세단독!FS5,=전세연립!FS5,=전세APT!GG5,=전세단독!GG5,=전세연립!GG5


,Seoul:Apartments(JDI),Seoul:Detached_Houses(JDI),Seoul:Row_Houses(JDI),Northern Seoul:Apartments(JDI),Northern Seoul:Detached_Houses(JDI),Northern Seoul:Row_Houses(JDI),Southern Seoul:Apartments(JDI),Southern Seoul:Detached_Houses(JDI),Southern Seoul:Row_Houses(JDI),6 Large Cities:Apartments(JDI),6 Large Cities:Detached_Houses(JDI),6 Large Cities:Row_Houses(JDI),Gyeonggi-do:Apartments(JDI),Gyeonggi-do:Detached_Houses(JDI),Gyeonggi-do:Row_Houses(JDI),Gangwon-do:Apartments(JDI),Gangwon-do:Detached_Houses(JDI),Gangwon-do:Row_Houses(JDI),Chungcheongbuk-do:Apartments(JDI),Chungcheongbuk-do:Detached_Houses(JDI),Chungcheongbuk-do:Row_Houses(JDI),Chungcheongnam-do:Apartments(JDI),Chungcheongnam-do:Detached_Houses(JDI),Chungcheongnam-do:Row_Houses(JDI),Jeollabuk-do:Apartments(JDI),Jeollabuk-do:Detached_Houses(JDI),Jeollabuk-do:Row_Houses(JDI),Jeollanam-do:Apartments(JDI),Jeollanam-do:Detached_Houses(JDI),Jeollanam-do:Row_Houses(JDI),Gyeongsangbuk-do:Apartments(JDI),Gyeongsangbuk-do:Detached_Houses(JDI),Gyeongsangbuk-do:Row_Houses(JDI),Gyeongsangnam-do:Apartments(JDI),Gyeongsangnam-do:Detached_Houses(JDI),Gyeongsangnam-do:Row_Houses(JDI),Non-Metropolitan Area:Apartments(JDI),Non-Metropolitan Area:Detached_Houses(JDI),Non-Metropolitan Area:Row_Houses(JDI)
0,=평균전세!I95,=평균전세!J95,=평균전세!K95,=평균전세!M95,=평균전세!N95,=평균전세!O95,=평균전세!Q95,=평균전세!R95,=평균전세!S95,=평균전세!U95,=평균전세!V95,=평균전세!W95,=평균전세!AU95,=평균전세!AV95,=평균전세!AW95,=평균전세!AY95,=평균전세!_95,=평균전세!_95,=평균전세!BA95,=평균전세!_95,=평균전세!_95,=평균전세!BC95,=평균전세!_95,=평균전세!_95,=평균전세!BE95,=평균전세!_95,=평균전세!_95,=평균전세!BG95,=평균전세!_95,=평균전세!_95,=평균전세!BI95,=평균전세!_95,=평균전세!_95,=평균전세!BK95,=평균전세!_95,=평균전세!_95,=평균전세!BM95,=평균전세!BN95,=평균전세!BO95


# Summary of Function

In [51]:
# Index to Return
def index_to_return(Target):
    log_return_df = []
    for col in Target.columns:
        log_return = []
        log_return = (np.log(Target[col]) - np.log(Target[col].shift(1)))
        log_return[0] = log_return.mean()
        log_return_df.append(log_return)
    log_return_final = pd.DataFrame(log_return_df).T
    return log_return_final

# Calulate Conditional Volatility
def conditional_volatility(Target):
    conditional_vol = []
    for col in Target.columns:
        Target_arima = sm.tsa.ARIMA(Target[col]*100, order=(1,0,1)).fit()
        Target_garch = arch.arch_model(Target_arima.resid.values, vol='Garch', p=1, o=0, q=1, dist='generalized error').fit(disp='off')
        conditional_vol.append(Target_garch.conditional_volatility)  
    conditional_vol_final = pd.DataFrame(conditional_vol, index=Target.columns, columns=Target.index).T
    return conditional_vol_final

# Rearrange of Stochastic Parameters
def get_stochastic_parameters(Target_index, Target_return, Target_cvol, deltat=1):
    parameters_final = dict()
    for col in Target_index.columns:
        parameters = pd.concat([Target_index[col], Target_return[col], Target_return[col].rolling(window=3, center=False).mean().fillna(0),
                                Target_cvol[col]/100], axis=1)
        parameters.columns = ['Index', 'Return', 'Mu', 'CVol']
        parameters['Nu'] = parameters['Mu'] - (0.5)*(parameters['CVol']**2)
        parameters['Up_return'] = np.exp(parameters['Nu']*deltat + parameters['CVol']*deltat)
        parameters['Down_return'] = np.exp(parameters['Nu']*deltat - parameters['CVol']*deltat)
        parameters_final[col] = parameters
    return parameters_final

# Grouping of Columns
def Grouping(S_real_full):
    Apartments_col = []
    Detached_Houses_col = []
    Row_Houses_col = []
    for col in S_real_full.keys():
        if 'Apartments(HPI)' in col.split(':'):
            Apartments_col.append(col)
        elif 'Detached_Houses(HPI)' in col.split(':'):
            Detached_Houses_col.append(col)
        elif 'Row_Houses(HPI)' in col.split(':'):
            Row_Houses_col.append(col)
    return Apartments_col, Detached_Houses_col, Row_Houses_col

# Calculate S
def S_generator(S_real, t, T):
    S = np.full((2**(T-t), T-t+1), np.nan)
    S[0][0] = S_real['Index'].iloc[t]
    for i in range(1,T-t+1): # 1~12 (2~13)
        for j in range(0,2**(i)): # 0~2^1-1/2^12-1 (1~2^1/2^12)
            if j%2 == 0:
                S[j][i] = S[int(j/2)][i-1] * S_real['Up_return'][t+(i-1)]
            else:
                S[j][i] = S[int(j/2)][i-1] * S_real['Down_return'][t+(i-1)]
    return S

# Calculate V
def V_generator(S_real, S, K_real, R_real, t, T, early_exercise):
    ### Generate V
    V_call = np.full((S.shape[0], S.shape[1]), np.nan)
    V_put = np.full((S.shape[0], S.shape[1]), np.nan)
    initial_spread = S_real['Index'][t] - K_real[t];
    V_call_early = np.full((S.shape[0], S.shape[1]-1), np.nan)
    V_put_early = np.full((S.shape[0], S.shape[1]-1), np.nan)
    Time_call_early = 0
    Time_put_early = 0
    print(T)
    for i in range(S.shape[0]-1, -1, -1): #4095~0 (4096~1)
        V_call[i][S.shape[1]-1] = S[i][S.shape[1]-1] - K_real[T] - initial_spread
        V_put[i][S.shape[1]-1] = S[i][S.shape[1]-1] - K_real[T] - initial_spread
    print(S[i][S.shape[1]-1])
    print(K_real[T])
    for j in range(0, S.shape[1]-1): #0~11 (1~12)
        for i in range(0, 2**(S.shape[1]-1-(j+1))): # 0~2^11-1/2^0(1~2^11/2^1)
            V_call[i][S.shape[1]-1-(j+1)] = (0.5 * V_call[2*i][S.shape[1]-1-j] + 0.5 * V_call[2*i+1][S.shape[1]-1-j]) / np.exp(R_real.iloc[T-j,0])
            V_put[i][S.shape[1]-1-(j+1)] = (0.5 * V_put[2*i][S.shape[1]-1-j] + 0.5 * V_put[2*i+1][S.shape[1]-1-j]) / np.exp(R_real.iloc[T-j,0])
            ### Calculate V(when early exercise is True)
            if early_exercise:
                V_call_early[i][S.shape[1]-1-(j+1)] = S[i][S.shape[1]-1-(j+1)] - K_real[T-(j+1)] - initial_spread
                if V_call_early[i][S.shape[1]-1-(j+1)] < V_call[i][S.shape[1]-1-(j+1)]:
                    V_call[i][S.shape[1]-1-(j+1)] = V_call_early[i][S.shape[1]-1-(j+1)]
                    Time_call_early = j+1
                V_put_early[i][S.shape[1]-1-(j+1)] = S[i][S.shape[1]-1-(j+1)] - K_real[T-(j+1)] - initial_spread
                if V_put_early[i][S.shape[1]-1-(j+1)] > V_put[i][S.shape[1]-1-(j+1)]:
                    V_put[i][S.shape[1]-1-(j+1)] = V_put_early[i][S.shape[1]-1-(j+1)]
                    Time_put_early = j+1
    return V_call, V_put, Time_call_early, Time_put_early

# Calculate V
def V_generator_simple(S_real, S, K_real, R_real, t, T, early_exercise):
    ### Generate V
    V_call = np.full((S.shape[0], S.shape[1]), np.nan)
    V_put = np.full((S.shape[0], S.shape[1]), np.nan)
    initial_spread = S_real['Index'][t] - K_real[t];
    V_early = np.full((S.shape[0], S.shape[1]-1), np.nan)
    Time_call_early = 0
    Time_put_early = 0
    for i in range(S.shape[0]-1, -1, -1): #4095~0 (4096~1)
        V_call[i][S.shape[1]-1] = S[i][S.shape[1]-1] - K_real[T] - initial_spread
        V_put[i][S.shape[1]-1] = S[i][S.shape[1]-1] - K_real[T] - initial_spread
    for j in range(0, S.shape[1]-1): #0~11 (1~12)
        for i in range(0, 2**(S.shape[1]-1-(j+1))): # 0~2^11-1/2^0(1~2^11/2^1)
            V_call[i][S.shape[1]-1-(j+1)] = (0.5 * V_call[2*i][S.shape[1]-1-j] + 0.5 * V_call[2*i+1][S.shape[1]-1-j]) / np.exp(R_real.iloc[T-j,0])
            V_put[i][S.shape[1]-1-(j+1)] = (0.5 * V_put[2*i][S.shape[1]-1-j] + 0.5 * V_put[2*i+1][S.shape[1]-1-j]) / np.exp(R_real.iloc[T-j,0])
            ### Calculate V(when early exercise is True)
            if early_exercise:
                V_early[i][S.shape[1]-1-(j+1)] = S[i][S.shape[1]-1-(j+1)] - K_real[T-(j+1)] - initial_spread
                if V_early[i][S.shape[1]-1-(j+1)] < V_call[i][S.shape[1]-1-(j+1)]:
                    V_call[i][S.shape[1]-1-(j+1)] = V_early[i][S.shape[1]-1-(j+1)]
                    Time_call_early = j+1
                if V_early[i][S.shape[1]-1-(j+1)] > V_put[i][S.shape[1]-1-(j+1)]:
                    V_put[i][S.shape[1]-1-(j+1)] = V_early[i][S.shape[1]-1-(j+1)]
                    Time_put_early = j+1
    return V_call, V_put, Time_call_early, Time_put_early

# Summary of One Column Spread
def Spread_OneColumn(S_real, K_real, R_real, start, end, interval, early_exercise):
    # Checking bound of input range
    initial = (pd.to_datetime(start).year - S_real.index[0].year) * 12 + pd.to_datetime(start).month - 1
    final = (pd.to_datetime(end).year - S_real.index[0].year) * 12 + pd.to_datetime(end).month - 1
    if (final - initial) < interval:
        print('Out of Bound!!!')
    # Calculate S and V
    V_final = []
    early_call = []
    early_put = []
    for t in range(initial, final, interval):
        T = t + interval
        ### Calculate S
        S = S_generator(S_real, t, T)
        ### Calculate V
        V_call, V_put, Time_call_early, Time_put_early = V_generator_simple(S_real, S, K_real, R_real, t, T, early_exercise)
        V_final.append([V_call[0][0], V_put[0][0]])
        ### Calculate dates of early exercises
        if Time_call_early == 0:
            early_call.append(0)
        else:
            early_call.append(S_real.index[T-Time_call_early])
        if Time_put_early == 0:
            early_put.append(0)
        else:
            early_put.append(S_real.index[T-Time_put_early])
    # Results
    Time_early_exercise = pd.concat([pd.DataFrame(early_call, columns=['Early Exercise of Call']),
                                     pd.DataFrame(early_put, columns=['Early Exercise of Put'])], axis=1)
    V_final = pd.DataFrame(V_final, columns=['V(Call)', 'V(Put)'])
    return V_final, Time_early_exercise

# Summary of Full Column Spread
def Spread_FullColumns(S_real_full, K_real_full, R_real, start, end, interval, early_exercise):
    Spread_full = dict()
    Time_early_full = dict()
    R_real = R_real.copy()
    for target in tqdm(S_real_full.keys()):
        S_real = S_real_full[target]
        K_real = K_real_full[target]
        Spread, Time_early = Spread_OneColumn(S_real, K_real, R_real, start, end, interval, early_exercise)
        Spread_full[target] = Spread
        Time_early_full[target] = Time_early
    return Spread_full, Time_early_full

# Summary of Full Result Statistics
def Summary_Result_Full(Result, Apartments_col, Detached_Houses_col, Row_Houses_col):
    print('Absolute Bubble Statistics in Full Period')
    result_summary = pd.DataFrame()
    for each in [Apartments_col, Detached_Houses_col, Row_Houses_col]:
        result_table = pd.concat([abs(Result[each]).mean(), abs(Result[each]).max(), 
                                  (abs(Result[each]).max()/abs(Result[each]).mean()-1)*100,
                                  abs(Result[each]).min(),
                                  (abs(Result[each]).min()/abs(Result[each]).mean()-1)*100], axis=1)
        result_table.columns = ['Average', 'Max', 'Max(%)', 'Min', 'Min(%)']
        result_summary = pd.concat([result_summary, pd.DataFrame(result_table.mean(), columns=[result_table.index[0].split(":")[1]]).T], axis=0)
        result_table = result_table.style.format({'Max(%)':'({:.0f}%)', 'Min(%)':'({:.0f}%)'})
        display(result_table)
    result_summary = result_summary.style.format({'Max(%)':'({:.0f}%)', 'Min(%)':'({:.0f}%)'})
    display(result_summary)
    print('Real Bubble Statistics in Full Period')
    result_summary = pd.DataFrame()
    for each in [Apartments_col, Detached_Houses_col, Row_Houses_col]:
        result_table = pd.concat([Result[each].mean(), Result[each].max(), Result[each].min(),
                            abs(Result[each].min()/Result[each].max())*100], axis=1)
        result_table.columns = ['Average', 'Max', 'Min', 'abs(Min/Max)(%)']
        result_summary = pd.concat([result_summary, pd.DataFrame(result_table.mean(), columns=[result_table.index[0].split(":")[1]]).T], axis=0)
        result_table = result_table.style.format({'abs(Min/Max)(%)':'({:.0f}%)'})
        display(result_table)  
    result_summary = result_summary.style.format({'abs(Min/Max)(%)':'({:.0f}%)'})
    display(result_summary)

# Summary of Trend Result Statistics
def Summary_Result_Trend(Result, Apartments_col, Detached_Houses_col, Row_Houses_col, windows=5):
    print('Absolute Bubble Statistics in Some periods')
    result_summary = pd.DataFrame()
    for each in [Apartments_col, Detached_Houses_col, Row_Houses_col]:
        result_table_concat = pd.DataFrame()
        for i in range(0,len(Result),windows):
            result_sub = Result.iloc[i:i+windows,:]
            result_table = pd.concat([abs(result_sub[each]).mean(), abs(result_sub[each]).max(), 
                                      (abs(result_sub[each]).max()/abs(result_sub[each]).mean()-1)*100,
                                      abs(result_sub[each]).min(),
                                      (abs(result_sub[each]).min()/abs(result_sub[each]).mean()-1)*100], axis=1)
            result_table_concat = pd.concat([result_table_concat, result_table], axis=1)
        result_summary = pd.concat([result_summary, pd.DataFrame(result_table_concat.mean(), columns=[result_table_concat.index[0].split(":")[1]]).T], axis=0)
        result_table_concat.columns = pd.MultiIndex.from_product([['2004~2008', '2009~2013', '2014~2018'],
                                                                  ['Average', 'Max', 'Max(%)', 'Min', 'Min(%)']])
        result_table_concat = result_table_concat.style.format({('2004~2008', 'Max(%)'):'({:.0f}%)', ('2004~2008', 'Min(%)'):'({:.0f}%)',
                                                                ('2009~2013', 'Max(%)'):'({:.0f}%)', ('2009~2013', 'Min(%)'):'({:.0f}%)',
                                                                ('2014~2018', 'Max(%)'):'({:.0f}%)', ('2014~2018', 'Min(%)'):'({:.0f}%)'})
        display(result_table_concat)
    result_summary.columns = pd.MultiIndex.from_product([['2004~2008', '2009~2013', '2014~2018'],
                                                         ['Average', 'Max', 'Max(%)', 'Min', 'Min(%)']])
    result_summary = result_summary.style.format({('2004~2008', 'Max(%)'):'({:.0f}%)', ('2004~2008', 'Min(%)'):'({:.0f}%)',
                                                  ('2009~2013', 'Max(%)'):'({:.0f}%)', ('2009~2013', 'Min(%)'):'({:.0f}%)',
                                                  ('2014~2018', 'Max(%)'):'({:.0f}%)', ('2014~2018', 'Min(%)'):'({:.0f}%)'})
    display(result_summary)
    print('Real Bubble Statistics in Full Period')
    result_summary = pd.DataFrame()
    for each in [Apartments_col, Detached_Houses_col, Row_Houses_col]:
        result_table_concat = pd.DataFrame()
        for i in range(0,len(Result),windows):
            result_sub = Result.iloc[i:i+windows,:]
            result_table = pd.concat([result_sub[each].mean(), result_sub[each].max(), result_sub[each].min(),
                                abs(result_sub[each].min()/result_sub[each].max())*100], axis=1)
            result_table_concat = pd.concat([result_table_concat, result_table], axis=1)
        result_summary = pd.concat([result_summary, pd.DataFrame(result_table_concat.mean(), columns=[result_table_concat.index[0].split(":")[1]]).T], axis=0)
        result_table_concat.columns = pd.MultiIndex.from_product([['2004~2008', '2009~2013', '2014~2018'],
                                                                  ['Average', 'Max', 'Min', 'abs(Min/Max)(%)']])
        result_table_concat = result_table_concat.style.format({('2004~2008', 'abs(Min/Max)(%)'):'({:.0f}%)',
                                                                ('2009~2013', 'abs(Min/Max)(%)'):'({:.0f}%)',
                                                                ('2014~2018', 'abs(Min/Max)(%)'):'({:.0f}%)'})
        display(result_table_concat)
    result_summary.columns = pd.MultiIndex.from_product([['2004~2008', '2009~2013', '2014~2018'],
                                                         ['Average', 'Max', 'Min', 'abs(Min/Max)(%)']])
    result_summary = result_summary.style.format({('2004~2008', 'abs(Min/Max)(%)'):'({:.0f}%)',
                                                  ('2009~2013', 'abs(Min/Max)(%)'):'({:.0f}%)',
                                                  ('2014~2018', 'abs(Min/Max)(%)'):'({:.0f}%)'})
    display(result_summary)

# Main
1) 만원단위 절삭: Index 나누기 10000  
2) Return 변환: (log(t)-log(t-1))으로 Return 변환 -> 첫줄(NaN)은 Return 평균값 대체  
3) Conditional Volatility 계산: Return*100으로 ARIMA(1,1) + GARCH(1,1) 적용  
4) Stochastic Parameters of S 정리:  
=> HPI / Return / Mu / CVol(위CVol나누기100) / Nu / Up_return / Down_return  
5) S 트리노미얼 트리 확장  
6) V 트리노미얼 트리 확장  
7) 조기행사 조건부 트리 업데이트  

In [17]:
# Data Loading
location = 'C:\\Users\\KK\\Desktop\\20190104_KB주택가격동향_KK.xlsx'
raw_HPI = pd.read_excel(location, sheet_name='Raw_HPI_value')
raw_JDI = pd.read_excel(location, sheet_name='Raw_JDI_value')
raw_IR = pd.read_excel(location, sheet_name='Raw_IR')
raw_IR.columns = ['Time', 'Interest_Rate']
raw_IR = raw_IR.iloc[3:,:]

# Indexing and Drop
raw_HPI.set_index(pd.date_range('1986-01', '2019-01', freq='M'), inplace=True)
raw_JDI.set_index(pd.date_range('1986-01', '2019-01', freq='M'), inplace=True)
raw_IR.set_index(pd.date_range('1987-01', '2019-01', freq='M'), inplace=True)
del raw_HPI['Time']
del raw_JDI['Time']
del raw_IR['Time']

# Time & Null Filter
raw_HPI = raw_HPI.loc[raw_HPI.index >= '2004-01', raw_HPI.isna().sum() == 0]
raw_JDI = raw_JDI.loc[raw_JDI.index >= '2004-01', raw_JDI.isna().sum() == 0]
raw_JDI.loc['2019-01-31'] = raw_JDI.iloc[len(raw_JDI)-1]
raw_IR = raw_IR[raw_IR.index >= '2004-01'] / (100*12) # Transfer to Monthly IR
raw_IR.loc['2019-01-31'] = raw_IR.values.flatten()[len(raw_IR)-1]

# Index to Return
raw_HPI_return = index_to_return(raw_HPI)

# Calulate Conditional Volatility
raw_HPI_cvol = conditional_volatility(raw_HPI_return)

# Rearrange of Stochastic Parameters
raw_HPI_parameters = get_stochastic_parameters(raw_HPI, raw_HPI_return, raw_HPI_cvol)

# Grouping of Columns
Apartments_col, Detached_Houses_col, Row_Houses_col = Grouping(raw_HPI_parameters)
    
# Calculate S & V value
# Spread, Time_early = Spread_OneColumn(raw_HPI_parameters['Seoul:Apartments(HPI)'], raw_JDI['Seoul:Apartments(HPI)'], raw_IR, 
#                                       start = '2004-01', end = '2018-12', interval = 12, early_exercise = False)
print('Processing of spread calculation without early exercise')
Spreadfull, Time_early_full = Spread_FullColumns(raw_HPI_parameters, raw_JDI, raw_IR, 
                                                 start = '2004-01', end = '2018-12', interval = 12, early_exercise = False)
print('Processing of spread calculation with early exercise')
Spreadfull_early, Time_early_full = Spread_FullColumns(raw_HPI_parameters, raw_JDI, raw_IR, 
                                                 start = '2004-01', end = '2018-12', interval = 12, early_exercise = True)

Processing of spread calculation without early exercise



100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [03:00<00:00,  7.19s/it]


Processing of spread calculation with early exercise



100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [03:50<00:00,  9.05s/it]


# Summary Result

In [52]:
result = pd.DataFrame()
for key in Spreadfull.keys():
    result = pd.concat([result, Spreadfull[key]['V(Call)']], axis=1)
result.columns = Spreadfull.keys()

result_call = pd.DataFrame()
for key in Spreadfull_early.keys():
    result_call = pd.concat([result_call, Spreadfull_early[key]['V(Call)']], axis=1)
result_call.columns = Spreadfull.keys()

result_put = pd.DataFrame()
for key in Spreadfull_early.keys():
    result_put = pd.concat([result_put, Spreadfull_early[key]['V(Put)']], axis=1)
result_put.columns = Spreadfull.keys()

# 전체 거품 기초통계
Summary_Result_Full(result, Apartments_col, Detached_Houses_col, Row_Houses_col)

# 5년 단위 거품 변화
Summary_Result_Trend(result, Apartments_col, Detached_Houses_col, Row_Houses_col, windows=5)

# 전체 거품 기초통계(조기행사로 거품줄어듬 정도 측정)
Summary_Result_Full(result_call, Apartments_col, Detached_Houses_col, Row_Houses_col)
Summary_Result_Full(result_put, Apartments_col, Detached_Houses_col, Row_Houses_col)

# 5년 단위 거품 변화(조기행사로 거품줄어듬 정도 측정)
Summary_Result_Trend(result_call, Apartments_col, Detached_Houses_col, Row_Houses_col, windows=5)
Summary_Result_Trend(result_put, Apartments_col, Detached_Houses_col, Row_Houses_col, windows=5)


Absolute Bubble Statistics in Full Period


,Average,Max,Max(%),Min,Min(%)
Seoul:Apartments(HPI),3290.35,9315.66,(183%),566.475,(-83%)
Northern Seoul:Apartments(HPI),2408.13,6202.18,(158%),197.541,(-92%)
Southern Seoul:Apartments(HPI),4055.16,12299.4,(203%),429.016,(-89%)
6 Large Cities:Apartments(HPI),325.732,1137.7,(249%),30.1186,(-91%)
Gyeonggi-do:Apartments(HPI),1331.85,3720.88,(179%),70.8455,(-95%)
Gangwon-do:Apartments(HPI),194.301,644.284,(232%),32.3033,(-83%)
Chungcheongbuk-do:Apartments(HPI),255.932,595.639,(133%),19.9111,(-92%)
Chungcheongnam-do :Apartments(HPI),185.204,596.522,(222%),11.0806,(-94%)
Jeollabuk-do:Apartments(HPI),231.232,1057.9,(358%),22.8853,(-90%)
Jeollanam-do:Apartments(HPI),139.804,276.265,(98%),31.4307,(-78%)


,Average,Max,Max(%),Min,Min(%)
Seoul:Detached_Houses(HPI),1586.41,3829.81,(141%),148.874,(-91%)
Northern Seoul:Detached_Houses(HPI),1462.74,3613.74,(147%),162.879,(-89%)
Southern Seoul:Detached_Houses(HPI),1764.47,4708.97,(167%),83.0129,(-95%)
6 Large Cities:Detached_Houses(HPI),247.818,740.99,(199%),53.8378,(-78%)
Gyeonggi-do:Detached_Houses(HPI),622.83,2326.3,(274%),52.9516,(-91%)
Non-Metropolitan Area:Detached_Houses(HPI),125.361,245.427,(96%),23.0936,(-82%)


,Average,Max,Max(%),Min,Min(%)
Seoul:Row_Houses(HPI),715.591,1808.22,(153%),55.4174,(-92%)
Northern Seoul:Row_Houses(HPI),642.456,1759.58,(174%),24.0378,(-96%)
Southern Seoul:Row_Houses(HPI),792.683,1816.61,(129%),54.7426,(-93%)
6 Large Cities:Row_Houses(HPI),137.388,564.236,(311%),24.0893,(-82%)
Gyeonggi-do:Row_Houses(HPI),389.989,1253.43,(221%),15.0611,(-96%)
Non-Metropolitan Area:Row_Houses(HPI),76.3489,188.948,(147%),14.7966,(-81%)


,Average,Max,Max(%),Min,Min(%)
Apartments(HPI),1016.5,3028.79,(228%),112.957,(-90%)
Detached_Houses(HPI),968.272,2577.54,(171%),87.4414,(-88%)
Row_Houses(HPI),459.076,1231.84,(189%),31.3575,(-90%)


...

,Average,Max,Min,abs(Min/Max)(%)
Apartments(HPI),640.269,3029.16,0,(0%)
Detached_Houses(HPI),871.945,2577.54,0,(0%)
Row_Houses(HPI),276.598,1230.18,0,(0%)


Absolute Bubble Statistics in Some periods


Real Bubble Statistics in Full Period


Absolute Bubble Statistics in Some periods


Real Bubble Statistics in Full Period
